# Notebook for Calculating 

Basic diagnostic on precipitation - enable comparison between two simulations

- Plot monthly mean precipitation and precipitation difference between simulation 1 & 2
- Compute seasonal cycle of precipitation at one location
- Compute seasonal cycle of precipitation over selected area

# Running the notebook
Execute the cells 1 by 1 by taping `ctrl + enter` or `shift + enter`

__Make sure to upload 2 file before continuing the rest of the notebook__

If you want to change the longitude and latitude bounds look in the data extraction section.

# Imports

In [ ]:
# Download new color palette
#Crameri : 
#palettable.scientific.sequential / palettable.scientific.diverging
#from palettable.scientific.sequential import Oslo_10
#cmap=Oslo_10.mpl_colormap
#https://jiffyclub.github.io/palettable/#documentation
!pip install palettable
from palettable.scientific.sequential import Davos_10_r, Oleron_15


from palettable.scientific.diverging import Vik_10


In [ ]:
# File upload widegt
from ipywidgets import FileUpload
from io import BytesIO

# Data manipulation and storage
import xarray as xr
import numpy as np

# Holoviews graphing
import hvplot.xarray
import holoviews as hv
from holoviews import opts

In [ ]:
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs
import cartopy.feature as cf

gv.extension('bokeh', 'matplotlib')

# Data Upload

In [ ]:
# Create file upload
# To load several files press shift with selecting - 
#all the files have to be selected at the same time.

file_upload = FileUpload(multiple=True)
file_upload

In [ ]:
# Put dataset into a list 
ds = []
for i in range(len(file_upload.data)):
    ds.append(xr.open_dataset(BytesIO(file_upload.data[i])))
names = list(file_upload.value.keys())

In [ ]:
#Scale precipitation to convert in mm/day
for dss in ds:
    dss['precip'] = dss['precip']*86400

In [ ]:
image = None 
for i in range(len(ds)):
    dss=ds[i]
    hist = dss[['precip', 'phis']].precip.hvplot(title=f"{names[i]}",
                                     width=300,
                                     height=300)
    if image is None:
        image=hist
    else:
        image=image+hist
image

In [ ]:
# Compute elevation from phis parameter
for i in range(len(ds)):
    ds[i]=ds[i].assign(alti=0)

for dfs in ds:
    dfs['alti'] = dfs['phis']/9.81
    
image=None
for i in range(len(ds)):
    dss=ds[i]
    hist = dss[['alti',]].alti.hvplot(title=f"{names[i]}",
                                     width=300,
                                     height=300)
    if image is None:
        image=hist
    else:
        image=image+hist
#image


# Plot Maps

Plot monthly mean precipitation for simulation 1 & 2

In [ ]:
d1 = 0
d2 = 1
# Plot monthly precipitations
topo = gv.Dataset(ds[d1]).to(gv.Image, ['lon','lat'],'alti')

opts.defaults(opts.Image(cmap=Davos_10_r.mpl_colormap,color_levels=10,clim=(0,20),frame_width=400,frame_height=200))

ensemble = gv.Dataset(ds[d1]).to(gv.Image, ['lon', 'lat'], 'precip')
ensemble1 = gv.Dataset(ds[d2]).to(gv.Image, ['lon', 'lat'], 'precip')
(ensemble.opts(
    title="Precipitation (mm/day)",projection=crs.Robinson())* 
 gv.operation.contours(topo,levels=[10]).opts(
     show_legend=False,cmap='gray_r',line_width=1.5) + 
 ensemble1.opts(
     colorbar=True,title="Precipitation (mm/day)",projection=crs.Robinson())*
 gv.operation.contours(topo,levels=[10]).opts(
     show_legend=False,cmap='gray_r',line_width=1.5))


Plot monthly mean precipitation anomaly between simulation 1 & 2

In [ ]:
# Compute difference between two simulations & zoom over specific region
difference = (ds[d1].precip - ds[d2].precip)

In [ ]:
# Plot differences
(gv.Dataset(difference).to(gv.Image, ['lon', 'lat'], 'precip').opts(
    width=600,height=300,
    cmap=Vik_10.mpl_colormap,color_levels=20,clim=(-15,15),
    colorbar=True,
    xlabel='Longitude',ylabel='Latitude',
    title='Precipitation anomaly (mm/day)',
    projection=crs.Robinson())*
 gv.operation.contours(topo,levels=[10]).opts(show_legend=False,cmap='gray_r',line_width=1.5))

# Plot seasonal cycles at one point

Here we first zoom on extended area just for esthetic issues

In [ ]:
# Select the desired zone
# /!\ Slice works based on the ordering in coordinates, here lat goes from +90 -> -90 and lon from -180 -> +180
dzone=[]

for dss in ds:
    dzone.append(dss.sel(lat=slice(40, -40), lon=slice(-180, 180)))
dzone

image=None
for i in range(2):
    dss=dzone[i]
    hist = gv.Dataset(dss).to(gv.Image, ['lon','lat'],'precip')
    if image is None:
        image=hist.opts(title=f"{names[i]}")
    else:
        image=image+hist.opts(title=f"{names[i]}")
image

Then we compute cycle at one location and plot

In [ ]:
ilat=-5
ilon=-30

image=None
for i in range(len(ds)):
    dss=dzone[i]
    hist_p = gv.Dataset(dss).to(gv.Image, ['lon','lat'],'precip')
    hist = gv.Dataset(dss).to(gv.Image, ['lon','lat'],'alti')
    cont=gv.operation.contours(hist,levels=[10])
    precip_curve = hv.Curve(hist_p.select(lon=ilon,lat=ilat), kdims=['time_counter']).opts(frame_width=400,frame_height=200)
    
    if image is None:
        image=  (hist_p * cont * gv.Points([(ilon,ilat)]) + precip_curve).opts(
        opts.Image(title="Precipitation (mm/day)"),
        opts.Points(color='k',size=5),
        opts.Contours(show_legend=False,line_width=1.5,cmap='gray_r',color_levels=0),
        opts.Curve(frame_width=400,frame_height=200))
    else:
        image=image*precip_curve
image


# Plot seasonal cycle over specific area

In [ ]:
ds_precip = ds[1][['precip', 'alti']]
ds1_precip = ds[0][['precip', 'alti']]


In [ ]:
# Select the desired zone
ulat = 10
ulon = -10
blat = -1
blon = -20

darea=[]

for dss in ds:
    darea.append(dss.sel(lat=slice(ulat, blat), lon=slice(blon, ulon)))


In [ ]:
# Select the desired zone
ulat = 10
ulon = -10
blat = -1
blon = -20

# /!\ Slice works based on the ordering in coordinates, here lat goes from +90 -> -90 and lon from -180 -> +180
ds_precip_zone = ds_precip.sel(lat=slice(ulat, blat), lon=slice(blon, ulon))
ds1_precip_zone = ds1_precip.sel(lat=slice(0, -20), lon=slice(-65, 0))

In [ ]:
# Plot precip over selected area
image=None
for i in range(len(ds)):
    dss=darea[i]
    dfs=dzone[i]
    hist_p = gv.Dataset(dfs).to(gv.Image, ['lon','lat'],'precip')
    continent= gv.Dataset(dfs).to(gv.Image, ['lon','lat'],'alti')
    hist = gv.Dataset(dss).to(gv.Image, ['lon','lat'],'alti')
    cont=gv.operation.contours(continent,levels=[10])
    precip_curve = hv.Curve(hist_p.select(lon=ilon,lat=ilat), kdims=['time_counter']).opts(frame_width=400,frame_height=200)
    
    if image is None:
        image=  (hist_p * cont * gv.Rectangles([(blon, blat,ulon,ulat)])).opts(
        opts.Image(title="Precipitation (mm/day)"),
        opts.Contours(show_legend=False,line_width=1.5,cmap='gray_r',color_levels=0),
        opts.Rectangles(color=None,line_width=3))
    else:
        image=image
image

In [ ]:
#darea[1].mean(dim=['lat', 'lon']).plot.line

In [ ]:

hist_p = gv.Dataset(dzone[1]).to(gv.Image, ['lon','lat'],'precip')
hist = gv.Dataset(dzone[1]).to(gv.Image, ['lon','lat'],'alti')

(hist_p * gv.operation.contours(hist,levels=[10]) * gv.Rectangles([(blon, blat,ulon,ulat)])).opts(
    opts.Image(title="Precipitation (mm/day)"),
    opts.Contours(show_legend=False,line_width=1.5,cmap='gray_r',color_levels=0),
    opts.Rectangles(color=None,line_width=3))



In [ ]:
 ds_precip_zone.precip.hvplot.image(label='precip').opts(cmap=Davos_10_r.mpl_colormap,color_levels=10,clim=(0,20)) 

In [ ]:
#Plot elevation
#ds_precip_zone.alti.hvplot.image(label='alti').opts(cmap=Oleron_15.mpl_colormap,color_levels=10,clim=(0,1000))

In [ ]:
# Extract values above a threshold
threshold = 1
upper = ds_precip_zone.where(ds_precip_zone.alti >= threshold)
upper1 = ds1_precip_zone.where(ds1_precip_zone.alti >= threshold)
# Extract values below threshold but above sea level
#lower = ds_precip_zone.where((ds_precip_zone.alti > 0) & (ds_precip_zone.alti < threshold))

In [ ]:
#Plot precip with threshold for elevation
#upper.hvplot.image().opts(cmap=Davos_10_r.mpl_colormap,color_levels=10,clim=(0,20))

# Calculate the mean value for each time step

In [ ]:
#upper.precip.mean(dim=['lat', 'lon']).plot.line()# + upper1.precip.mean(dim=['lat', 'lon']).plot.line()

In [ ]:
#lower.hvplot.image()

In [ ]:
#lower.precip.mean(dim=['lat', 'lon']).plot.line();

In [ ]:
#lower.precip.sum(dim=['lat', 'lon']).plot() + upper.precip.sum(dim=['lat', 'lon']).plot();

# Calculate the sum over all locations at each time step

In [ ]:
#upper.precip.sum(dim=['lat', 'lon']).plot.line();

# Calculate the cumlative sum for each time step

In [ ]:
#upper.precip.sum(dim=['lat', 'lon']).cumsum().plot.line();

In [ ]:
#lower.precip.sum(dim=['lat', 'lon']).cumsum().plot.line();